In [1]:
from argparse import ArgumentParser
# from gym import make
# from gym.spaces import Box, Discrete
# import gym
# import gym_fairrec
# import roboschool
# from yaml import load
# import yaml

from models import build_diag_gauss_policy, build_mlp, build_multinomial_policy
# from simulators import *
from transforms import *
from torch_utils import get_device
from trpo import TRPO

import pandas as pd
import numpy as np
# from read_data import read_file, read_embeddings, Embeddings

from data_util import read_file
from fair_env import *
# from env import *
# from ddpg import *
import torch
import os
import time
import torch.nn.functional as F

In [2]:
model_name = 'fairrec'
data_name = 'ml-1m'
data = read_file('./data/'+data_name+'/train_data.csv')
item_embeddings = np.load('./data/'+data_name+'/item_embed.npy')
user_embeddings = np.load('./data/'+data_name+'/user_embed.npy')
item_indicator = np.load('./data/'+data_name+'/item_indicator_28.npy')


nb_item = item_embeddings.shape[0]
nb_user = user_embeddings.shape[0]
print('num of users: %d, num of items: %d' %(nb_user, nb_item))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


env_args = {}
env_args['data'] = data
env_args['nb_user'] = nb_user
env_args['nb_item'] = nb_item
env_args['item_embeddings'] = item_embeddings
env_args['user_embeddings'] = user_embeddings
env_args['item_indicator'] = item_indicator
env_args['device'] = device
env_args['gamma'] = 0.95

env = Environment(**env_args)
print('Successfully create Training Env!')

num of users: 6040, num of items: 3952
Successfully create Training Env!


In [3]:
history_length = 5 # N in article
ra_length = 1 # K in article
state_space_size = item_embeddings.shape[1] * history_length
action_space_size = item_embeddings.shape[1] * ra_length


vf_hidden_dims = [64]
vf_args = (state_space_size + 1, vf_hidden_dims, 1)
value_fun = build_mlp(*vf_args)

policy_hidden_dims = [64]
policy_args = (state_space_size, policy_hidden_dims, action_space_size)
policy = build_diag_gauss_policy(*policy_args)


policy.to(device)
value_fun.to(device)
print(policy)
print(value_fun)

Sequential(
  (0): Linear(in_features=500, out_features=64, bias=True)
  (1): Tanh()
  (2): Linear(in_features=64, out_features=100, bias=True)
  (3): DiagGaussianLayer()
)
Sequential(
  (0): Linear(in_features=501, out_features=64, bias=True)
  (1): Tanh()
  (2): Linear(in_features=64, out_features=1, bias=True)
)


In [4]:
from collections import defaultdict
import gym
from gym.spaces import Box, Discrete
import numpy as np
import torch
# from torch_utils import get_device


class Simulator:
    def __init__(self, env, policy, n_trajectories, trajectory_len, **env_args):
        self.env = env
        self.policy = policy


class SinglePathSimulator(Simulator):
    def __init__(self, env, policy, n_trajectories, trajectory_len, **env_args):
        Simulator.__init__(self, env, policy, n_trajectories, trajectory_len, **env_args)
        self.item_embeddings= env_args['item_embeddings']
        self.trajectory_len = trajectory_len
        self.n_trajectories = n_trajectories
        self.nb_item = env_args['nb_item']
        self.device = env_args['device']

    def sample_trajectories(self):
        self.policy.eval()

        with torch.no_grad():
            memory = np.asarray([defaultdict(list) for i in range(self.n_trajectories)])
        #     done = [False] * n_trajectories

            ra_length = 1
#             len_trajectory = 10
#             epsilon = 0.9
            item_embeds = torch.from_numpy(self.item_embeddings).to(self.device).float()

#             memory_states = []
#             memory_actions = []
#             memory_rewards = []
#             memory_done = []

            ave_score = 0
            ave_cost = 0
            states = self.env.reset()
            recommended_item_onehot = torch.FloatTensor(self.n_trajectories, self.nb_item).zero_().to(device)
            recommendations = []
            for t in range(self.trajectory_len): 
                policy_input = torch.FloatTensor(states).to(self.device).view(self.n_trajectories, -1)
                weight_dists = self.policy(policy_input)
                w = weight_dists.sample()
                item_weights = torch.mm(w.view(-1,item_embeds.shape[1]), item_embeds.transpose(0,1)).view(self.n_trajectories, ra_length, -1)
                item_weights = torch.mul(item_weights.transpose(0,1), 1-recommended_item_onehot).reshape(states.shape[0],ra_length,-1)
                item_idxes = torch.argmax(item_weights,dim=2)

                recommendations.append(item_idxes)
                recommended_item_onehot = recommended_item_onehot.scatter_(1, item_idxes, 1)

                actions = item_embeds[item_idxes.cpu().detach()]
                states_prime, rewards, costs, info = self.env.step(actions, item_idxes)

        #         states_prime, rewards, info = env.step(item_idxes)
        #         memory_states.append(policy_input)
        #         memory_actions.append(actions)
        #         memory_rewards.append(rewards)
        #         memory_done.append(done)

                for i in range(len(memory)):
                    trajectory = memory[i]
                    trajectory['states'].append(policy_input[i].to(device).squeeze())
                    trajectory['actions'].append(actions[i].to(device).squeeze())
                    trajectory['rewards'].append(rewards[i].to(device).squeeze())


                states = states_prime
                ave_score += torch.sum(info).detach().cpu()
                ave_cost += torch.sum(costs).detach().cpu()
            for trajectory in memory:
                trajectory['done'] = True    
            print(ave_score/self.trajectory_len, ave_cost/self.trajectory_len)
            print(torch.cat(recommendations,1))
            return memory

In [5]:
n_trajectories = nb_user
trajectory_len = 10
simulator = SinglePathSimulator(env, policy, n_trajectories, trajectory_len, **env_args)

In [6]:
try:
    trpo_args = config['trpo_args']
except:
    trpo_args = {}

trpo = TRPO(policy, value_fun, simulator, model_name=model_name,
            continue_from_file=False, **trpo_args)

In [7]:
n_episodes = 10
trpo.train(n_episodes)

tensor(1890) tensor(1030.3000)
tensor([[2382,  662, 2654,  ..., 2852, 2853, 3820],
        [1492, 3645,  809,  ..., 1706, 1896, 2494],
        [3862, 1273, 2482,  ..., 2605, 3184, 2695],
        ...,
        [3162,  448, 3659,  ..., 2505, 3554, 1333],
        [3027,  117, 2582,  ...,  233, 1720,  875],
        [3689, 1957, 1088,  ..., 1240, 2559, 1349]], device='cuda:0')
[EPISODE]: 1	[AVG. REWARD]: 3.1293	 [ELAPSED TIME]: 0:00:10
tensor(1969) tensor(1050.1000)
tensor([[2300,  817, 2313,  ..., 2330, 1335, 3819],
        [1494, 3867, 2360,  ..., 2458, 1923, 1957],
        [2494, 1940, 1600,  ..., 3264, 2116, 1393],
        ...,
        [ 212, 2642,  174,  ..., 3326, 2492,  122],
        [1983,  601,   85,  ..., 3216, 2548,  444],
        [ 239, 2426,   61,  ...,  321,  215, 2317]], device='cuda:0')
[EPISODE]: 2	[AVG. REWARD]: 3.2608	 [ELAPSED TIME]: 0:00:21
tensor(2044) tensor(1088.3000)
tensor([[3527, 3158,  464,  ..., 3862, 1688, 3006],
        [2336,  122, 3445,  ..., 3073,  713, 1231